<h1>Pipeline</h1>

In [13]:
import numpy as np
import pandas as pd

from importlib import reload

%matplotlib inline
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.52 ms (started: 2021-09-07 10:30:13 +02:00)


In [25]:
from src.data import data as data
from src.preprocessing import preprocessing as preprocess
from src.models import model as model
from src.visualization import visualize as vis

time: 406 µs (started: 2021-09-07 10:39:39 +02:00)


In [14]:
reload(data)

data_path = "../data/kddcup.data.gz"
df = data.load_data(data_path)

time: 13.5 s (started: 2021-09-07 10:30:18 +02:00)


In [19]:
reload(preprocess)

df = preprocess.features_encoder(df)

time: 4.53 s (started: 2021-09-07 10:34:08 +02:00)


In [28]:
reload(vis)

print(vis.memory(df), "megabytes")

1508.72 megabytes
time: 2.41 ms (started: 2021-09-07 10:40:52 +02:00)


In [32]:
from sklearn.model_selection import train_test_split
reload(model)

train, test = train_test_split(df, test_size=0.3, random_state=0)
clf = model.train_model(train)

time: 3.28 s (started: 2021-09-07 10:51:36 +02:00)


In [36]:
reload(vis)

x_test = test.drop(["label"], axis="columns")
y_test = test["label"]
pred = clf.predict(x_test)

print(vis.accuracy(y_test, pred, True))

Correct: 1469465
Incorrect: 65
0.9999557681707757
time: 10.9 s (started: 2021-09-07 10:56:39 +02:00)
